# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = os.path.join(MAIN_PATH,'data')
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
SEED = 42
torch.manual_seed(42)
np.random.seed(42)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc

# Evaluate logits

In [5]:
DATASET = 'ImageNet'
VAL_SIZE = 0.1 #5000
SUB_VAL_SIZE = 0.2
METRIC = metrics.AURC
method = measures.max_logit

In [6]:
p_range = range(10)

d_naurc_p = {p:defaultdict(list) for p in p_range}
d_naurc_msp_p = {p:defaultdict(list) for p in p_range}
d_naurc_opt = defaultdict(list)
d_naurc_msp_opt = defaultdict(list)
d_baseline = defaultdict(list)
acc = defaultdict(list)

In [7]:
NUM_SPLITS = 10
seed = SEED
for i in range(NUM_SPLITS):
    for model_arc in models.list_models(DATASET):
        #print(model_arc)
        with torch.no_grad():
            logits_val,labels_val,logits_test,labels_test = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                split = 'test', device = dev),VAL_SIZE,seed = seed)
            logits_val,labels_val = logits_val[:int(SUB_VAL_SIZE*labels_val.size(0))],labels_val[:int(SUB_VAL_SIZE*labels_val.size(0))]
            risk_val = measures.wrong_class(logits_val,labels_val).float()
            risk_test = measures.wrong_class(logits_test,labels_test).float()
        acc[model_arc].append(1-risk_test.mean().item())

        for p in p_range:
            d_naurc_p[p][model_arc].append(metrics.N_AURC(method(post_hoc.normalize(logits_test,p)),risk_test))
            p,T = post_hoc.optimize.p_and_T(logits_val,risk_val,p_range=[p])
            d_naurc_msp_p[p][model_arc].append(metrics.N_AURC(measures.MSP(post_hoc.normalize(logits_test,p).div(T)),risk_test))
        p = post_hoc.optimize.p(logits_val,risk_val,metric = METRIC)
        d_naurc_opt[model_arc].append(metrics.N_AURC(post_hoc.MaxLogit_pNorm(logits_test,p),risk_test))
        pT = post_hoc.optimize.p_and_T(logits_val,risk_val,metric = METRIC)
        d_naurc_msp_opt[model_arc].append(metrics.N_AURC(measures.MSP(post_hoc.normalize(logits_test,pT[0]).div(pT[1])),risk_test))
        d_baseline[model_arc].append(metrics.N_AURC(measures.MSP(logits_test),risk_test))
    seed = seed+10
    models_list = list(acc.keys())

In [8]:
naurc_means = {}
naurc_std = {}
baseline_means = {}
for m,v in d_naurc_opt.items():
    naurc_means[m] = np.mean(v)
    baseline_means[m] = np.mean(d_baseline[m])
    naurc_std[m] = np.std(v)

In [9]:
gains = {}
apg = {}
gains_msp = {}
apg_msp = {}
apg_std = {}
apg_msp_std = {}
for p in p_range:
    gains[p] = post_hoc.significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_p[p].values())))
    gains_msp[p] = post_hoc.significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_msp_p[p].values())))
gains['Tunable'] = post_hoc.significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_opt.values())))
gains_msp['Tunable'] = post_hoc.significant(np.array(list(d_baseline.values()))-np.array(list(d_naurc_msp_opt.values())))
for p in gains:
    apg[p] = gains[p].mean()
    apg_msp[p] = gains_msp[p].mean()
    apg_std[p] = gains[p].mean(0).std()
    apg_msp_std[p] = gains_msp[p].mean(0).std()

In [10]:
post_hoc.significant((np.array(list(d_baseline.values()))-np.array(list(d_naurc_opt.values())))).mean()

0.0679490162827848

In [11]:
print(r'$p$ & MaxLogit-pNorm & MSP-pNorm \\')
print('\midrule')
for p in apg.keys():
    string = f'{p}'
    string += f"& {apg[p]:.5f}"+r' {\footnotesize $\pm$'+f"{apg_std[p]:.5f}" + "}"
    string += f"& {apg_msp[p]:.5f}"+r' {\footnotesize $\pm$'+f"{apg_msp_std[p]:.5f}" + "}"
    print(string + r' \\')

$p$ & MaxLogit-pNorm & MSP-pNorm \\
\midrule
0& 0.00000 {\footnotesize $\pm$0.00000}& 0.05571 {\footnotesize $\pm$0.00164} \\
1& 0.00199 {\footnotesize $\pm$0.00007}& 0.05699 {\footnotesize $\pm$0.00365} \\
2& 0.01519 {\footnotesize $\pm$0.00050}& 0.06234 {\footnotesize $\pm$0.00329} \\
3& 0.05058 {\footnotesize $\pm$0.00049}& 0.06527 {\footnotesize $\pm$0.00340} \\
4& 0.06443 {\footnotesize $\pm$0.00051}& 0.06621 {\footnotesize $\pm$0.00375} \\
5& 0.06805 {\footnotesize $\pm$0.00048}& 0.06625 {\footnotesize $\pm$0.00338} \\
6& 0.06814 {\footnotesize $\pm$0.00048}& 0.06589 {\footnotesize $\pm$0.00332} \\
7& 0.06692 {\footnotesize $\pm$0.00053}& 0.06551 {\footnotesize $\pm$0.00318} \\
8& 0.06544 {\footnotesize $\pm$0.00048}& 0.06512 {\footnotesize $\pm$0.00345} \\
9& 0.06410 {\footnotesize $\pm$0.00048}& 0.06491 {\footnotesize $\pm$0.00329} \\
Tunable& 0.06795 {\footnotesize $\pm$0.00077}& 0.06436 {\footnotesize $\pm$0.00413} \\
